# Example 5: Query EBUcore Metadata with fragments

In [1]:
import sys
from tqdm.notebook import tnrange
from sgqlc.operation import Operation
from pdp_schema import pdp_schema
from ebucore_schema import ebucore_schema
from pdp_graphql_client_python import client

## 1. Set the API endpoint
Ensure the endpoint is set to new EBUcore schema on DEV

In [2]:
%set_env PDP_API=https://graphql-api.pdp.dev.srgssr.ch/graphql

env: PDP_API=https://graphql-api.pdp.dev.srgssr.ch/graphql


## 2. Configure the query operation

In [3]:
# use example
op = Operation(ebucore_schema.Query)
assets = op.assets(ids=[
    "30115005-A6C3-4708-98F8-10FB082E381E",
    "7296F1FD-5767-4BB9-9C3C-546959723141",
    "urn:srf:video:271310e9-f391-4d28-8495-be660fce42f1"])

In [4]:
op

query {
  assets(ids: ["30115005-A6C3-4708-98F8-10FB082E381E", "7296F1FD-5767-4BB9-9C3C-546959723141", "urn:srf:video:271310e9-f391-4d28-8495-be660fce42f1"]) {
    assetId
    title
    abstract
    date
    hasContributor {
      agentName
      hasRole
    }
  }
}

## 3. Select fragment queries with `__as__` statement

In [5]:
# define fragements
from ebucore_schema import Series, Episode, Staff

assets.asset_id()
assets.title()
assets.__as__(Series).total_number_of_episodes() # => if asset is of instance Series, retrieve the number of episodes
assets.__as__(Episode).orientation()
assets.has_contributor.__as__(Staff).given_name()
assets.has_contributor.__as__(Staff).family_name()

familyName

In [6]:
op

query {
  assets(ids: ["30115005-A6C3-4708-98F8-10FB082E381E", "7296F1FD-5767-4BB9-9C3C-546959723141", "urn:srf:video:271310e9-f391-4d28-8495-be660fce42f1"]) {
    assetId
    title
    __typename
    hasContributor {
      __typename
      ... on Staff {
        givenName
        familyName
      }
    }
    ... on Series {
      totalNumberOfEpisodes
    }
    ... on Episode {
      orientation
    }
  }
}

## 4. Retrieve the data

In [7]:
# query items
data = client.run_query(op)

In [8]:
data

{'data': {'assets': [{'assetId': '30115005-a6c3-4708-98f8-10fb082e381e',
    '__typename': 'Episode',
    'hasContributor': [{'__typename': 'Staff',
      'givenName': 'Béatrice',
      'familyName': 'Mohr'},
     {'__typename': 'Staff', 'givenName': 'Kurt', 'familyName': 'Schaad'}],
    'orientation': None,
    'title': 'Fernweh: Von Kapstadt zu den Viktoriafällen'},
   {'assetId': '7296f1fd-5767-4bb9-9c3c-546959723141',
    '__typename': 'Series',
    'title': 'Die Strassenfliege',
    'totalNumberOfEpisodes': '5',
    'hasContributor': []},
   {'assetId': 'urn:srf:video:271310e9-f391-4d28-8495-be660fce42f1',
    '__typename': 'Episode',
    'hasContributor': [{'__typename': 'Staff',
      'givenName': 'Ueli',
      'familyName': 'Schmezer'}],
    'orientation': None,
    'title': 'Billiger Echtpelz. Knausrige Samsung. Winterhandschuh-Test.'}]}}

## 5. Transform JSON return data to python object

In [9]:
results = (op + data).assets

In [10]:
results

[Episode(asset_id='30115005-a6c3-4708-98f8-10fb082e381e', title='Fernweh: Von Kapstadt zu den Viktoriafällen', __typename__='Episode', has_contributor=[Staff(__typename__='Staff', given_name='Béatrice', family_name='Mohr'), Staff(__typename__='Staff', given_name='Kurt', family_name='Schaad')], orientation=None),
 Series(asset_id='7296f1fd-5767-4bb9-9c3c-546959723141', title='Die Strassenfliege', __typename__='Series', has_contributor=[], total_number_of_episodes='5'),
 Episode(asset_id='urn:srf:video:271310e9-f391-4d28-8495-be660fce42f1', title='Billiger Echtpelz. Knausrige Samsung. Winterhandschuh-Test.', __typename__='Episode', has_contributor=[Staff(__typename__='Staff', given_name='Ueli', family_name='Schmezer')], orientation=None)]

## 6. Filter objects by fragment type

In [11]:
for result in results:
    if isinstance(result, Episode):
        contributor_names = ','.join([x.given_name+' '+x.family_name for x in result.has_contributor])
        print(f'Episode "{result.title}" has contributors {contributor_names})')
    if isinstance(result, Series):
        print(f'Series "{result.title}" has {result.total_number_of_episodes} episodes')

Episode "Fernweh: Von Kapstadt zu den Viktoriafällen" has contributors Béatrice Mohr,Kurt Schaad)
Series "Die Strassenfliege" has 5 episodes
Episode "Billiger Echtpelz. Knausrige Samsung. Winterhandschuh-Test." has contributors Ueli Schmezer)


## 7. Load data into a `pandas` dataframe

In [12]:
import pandas as pd

In [13]:
df = pd.DataFrame([x.__json_data__ for x in results])

In [14]:
df

,assetId,__typename,hasContributor,orientation,title,totalNumberOfEpisodes
0,30115005-a6c3-4708-98f8-10fb082e381e,Episode,"[{'givenName': 'Béatrice', 'familyName': 'Mohr...",NaN,Fernweh: Von Kapstadt zu den Viktoriafällen,NaN
1,7296f1fd-5767-4bb9-9c3c-546959723141,Series,[],NaN,Die Strassenfliege,5
2,urn:srf:video:271310e9-f391-4d28-8495-be660fce...,Episode,"[{'givenName': 'Ueli', 'familyName': 'Schmezer'}]",NaN,Billiger Echtpelz. Knausrige Samsung. Winterha...,NaN


In [15]:
# explode has_Contributor array
df2 = df.explode('hasContributor')

In [16]:
df2

,assetId,__typename,hasContributor,orientation,title,totalNumberOfEpisodes
0,30115005-a6c3-4708-98f8-10fb082e381e,Episode,"{'givenName': 'Béatrice', 'familyName': 'Mohr'}",NaN,Fernweh: Von Kapstadt zu den Viktoriafällen,NaN
0,30115005-a6c3-4708-98f8-10fb082e381e,Episode,"{'givenName': 'Kurt', 'familyName': 'Schaad'}",NaN,Fernweh: Von Kapstadt zu den Viktoriafällen,NaN
1,7296f1fd-5767-4bb9-9c3c-546959723141,Series,NaN,NaN,Die Strassenfliege,5
2,urn:srf:video:271310e9-f391-4d28-8495-be660fce...,Episode,"{'givenName': 'Ueli', 'familyName': 'Schmezer'}",NaN,Billiger Echtpelz. Knausrige Samsung. Winterha...,NaN


In [17]:
pd.json_normalize(data = df2.loc[df2.__typename == 'Episode'].hasContributor, meta=['assetId'])

,givenName,familyName
0,Béatrice,Mohr
1,Kurt,Schaad
2,Ueli,Schmezer
